In [1]:
import logging
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2,3,4,5'
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import json
from constants_prompt import build_autoj_input

model_path_list = ['/root/Qwen1.5-7B-Chat', '/root/Meta-Llama-3-8B-Instruct', '/root/.cache/modelscope/hub/qwen/Qwen1___5-14B-Chat',
                   '/root/Nanbeige2-8B-Chat', '/root/.cache/modelscope/hub/ZhipuAI/chatglm3-6b', 
                   '/root/.cache/modelscope/hub/Shanghai_AI_Laboratory/internlm2-chat-7b',
                   '/root/.cache/modelscope/hub/deepseek-ai/deepseek-llm-7b-chat',
                   '/root/.cache/modelscope/hub/lockonlvange/autoj-13b-fp16',
                   '/root/PandaLM-7B-v1']

In [2]:
model_name_or_path = model_path_list[3]
llm = LLM(model=model_name_or_path, tensor_parallel_size=4, trust_remote_code=True, tokenizer_mode="auto", dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
sampling_params = SamplingParams(temperature=0.0, top_p=1.0, max_tokens=16384)
model_name = model_name_or_path.rsplit('/', 1)[-1]
from utils import extract_pariwise_result

2024-05-29 11:19:14,903	WARNING services.py:2009 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 1073741824 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2024-05-29 11:19:16,089	INFO worker.py:1749 -- Started a local Ray instance.


INFO 05-29 11:19:17 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='/root/PandaLM-7B-v1', speculative_config=None, tokenizer='/root/PandaLM-7B-v1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=/root/PandaLM-7B-v1)


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


INFO 05-29 11:19:26 utils.py:660] Found nccl from library /root/.config/vllm/nccl/cu12/libnccl.so.2.18.1
(RayWorkerWrapper pid=2103410) INFO 05-29 11:19:26 utils.py:660] Found nccl from library /root/.config/vllm/nccl/cu12/libnccl.so.2.18.1
(RayWorkerWrapper pid=2103410) INFO 05-29 11:19:27 selector.py:27] Using FlashAttention-2 backend.
INFO 05-29 11:19:27 selector.py:27] Using FlashAttention-2 backend.
INFO 05-29 11:19:29 pynccl_utils.py:43] vLLM is using nccl==2.18.1
(RayWorkerWrapper pid=2103410) INFO 05-29 11:19:29 pynccl_utils.py:43] vLLM is using nccl==2.18.1
WARNING 05-29 11:19:29 custom_all_reduce.py:65] Custom allreduce is disabled because it's not supported on more than two PCIe-only GPUs. To silence this warning, specify disable_custom_all_reduce=True explicitly.
(RayWorkerWrapper pid=2103410) WARNING 05-29 11:19:29 custom_all_reduce.py:65] Custom allreduce is disabled because it's not supported on more than two PCIe-only GPUs. To silence this warning, specify disable_custo

In [3]:
# autoj
texts_autoj = []
input_file_path = '/root/auto-j/data/test/testdata_pairwise.jsonl'
with open(input_file_path, 'r') as f:
    for line in f:
        item = json.loads(line)
        prompt = build_autoj_input(prompt=item['prompt'], resp1=item['response 1'], resp2=item['response 2'], protocol="pairwise_tie")
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_autoj.append(text)
outputs_autoj = llm.generate(texts_autoj, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_response.json'
with open(input_file_path, 'r') as fin:
    with open(output_file_path, 'w') as f:
        for output, line in zip(outputs_autoj, fin):
            item = json.loads(line)
            pred_label = extract_pariwise_result(output.outputs[0].text)
            item['response_judge'] = output.outputs[0].text
            item['pred_label'] = pred_label
            json.dump(item, f)
            f.write('\n')
            print(pred_label)

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.
Token indices sequence length is longer than the specified maximum sequence length for this model (1496 > 1024). Running this sequence through the model will result in indexing errors
Processed prompts:   3%|▎         | 44/1392 [00:06<03:22,  6.65it/s]

WARNING 05-29 11:20:10 scheduler.py:648] Input prompt (3043 tokens) is too long and exceeds limit of 2048
WARNING 05-29 11:20:10 scheduler.py:648] Input prompt (2650 tokens) is too long and exceeds limit of 2048


Processed prompts:   4%|▎         | 49/1392 [00:06<02:41,  8.33it/s]

WARNING 05-29 11:20:11 scheduler.py:648] Input prompt (4579 tokens) is too long and exceeds limit of 2048
WARNING 05-29 11:20:11 scheduler.py:648] Input prompt (4007 tokens) is too long and exceeds limit of 2048


Processed prompts:   4%|▍         | 53/1392 [00:07<02:35,  8.59it/s]

...

WARNING 05-29 11:23:10 scheduler.py:648] Input prompt (2545 tokens) is too long and exceeds limit of 2048


Processed prompts:  99%|█████████▉| 1385/1392 [03:08<00:01,  4.59it/s]

WARNING 05-29 11:23:12 scheduler.py:648] Input prompt (3052 tokens) is too long and exceeds limit of 2048


Processed prompts: 100%|██████████| 1392/1392 [03:09<00:00,  7.33it/s]

-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-

In [4]:
# pandalm
prompt_templates = {
    "alpaca": {
        "description": "Template used by Alpaca-LoRA.",
        "prompt_input": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",
        "prompt_no_input": "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n",
        "response_split": "### Response:",
    }
}
def get_prompt(instruction, input):
    prompt = prompt_templates['alpaca']["prompt_input"].format(instruction=instruction, input=input)
    return prompt


texts_pandalm = []
    
import json
input_file_path = '/root/PandaLM/data/testset-v1.json'
with open(input_file_path, 'r') as f:
    input_data_pandalm = json.load(f)
    for item in input_data_pandalm:
        prompt = build_autoj_input(prompt=get_prompt(item['instruction'], item['input']), resp1=item['response1'], resp2=item['response2'], protocol="pairwise_tie")
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_pandalm.append(text)

outputs_pandalm = llm.generate(texts_pandalm, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_response.json'
with open(output_file_path, 'w') as f:
    for output, item in zip(outputs_pandalm, input_data_pandalm):
        pred_label = extract_pariwise_result(output.outputs[0].text)
        item['response_judge'] = output.outputs[0].text
        item['pred_label'] = pred_label
        json.dump(item, f)
        f.write('\n')
        print(pred_label)

Processed prompts: 100%|██████████| 999/999 [01:26<00:00, 11.60it/s]


-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-

In [5]:
# llmbar_natural
texts_natural = []
input_file_path = '/root/LLMBar/Dataset/LLMBar/Natural/dataset.json'
with open(input_file_path, 'r') as f:
    input_data_llmbar_natural = json.load(f)
    for item in input_data_llmbar_natural:
        prompt = build_autoj_input(prompt=item['input'], resp1=item['output_1'], resp2=item['output_2'], protocol="pairwise_tie")
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_natural.append(text)
outputs_llmbar_natural = llm.generate(texts_natural, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_response.json'
with open(output_file_path, 'w') as f:
    for output, item in zip(outputs_llmbar_natural, input_data_llmbar_natural):
        pred_label = extract_pariwise_result(output.outputs[0].text)
        item['response_judge'] = output.outputs[0].text
        item['pred_label'] = pred_label
        json.dump(item, f)
        f.write('\n')
        print(pred_label)

Processed prompts: 100%|██████████| 100/100 [00:08<00:00, 12.08it/s]

-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1


In [6]:
# llmbar_neighbor

texts_neighbor = []
input_file_path = '/root/LLMBar/Dataset/LLMBar/Adversarial/Neighbor/dataset.json'
with open(input_file_path, 'r') as f:
    input_data_llmbar_neighbor = json.load(f)
    for item in input_data_llmbar_neighbor:
        prompt = build_autoj_input(prompt=item['input'], resp1=item['output_1'], resp2=item['output_2'], protocol="pairwise_tie")
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_neighbor.append(text)
outputs_llmbar_neighbor = llm.generate(texts_neighbor, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_response.json'
with open(output_file_path, 'w') as f:
    for output, item in zip(outputs_llmbar_neighbor, input_data_llmbar_neighbor):
        pred_label = extract_pariwise_result(output.outputs[0].text)
        item['response_judge'] = output.outputs[0].text
        item['pred_label'] = pred_label
        json.dump(item, f)
        f.write('\n')
        print(pred_label)

Processed prompts: 100%|██████████| 134/134 [00:11<00:00, 11.81it/s]

-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1


In [7]:
# llmbar_gptinst

texts_gptinst = []
input_file_path = '/root/LLMBar/Dataset/LLMBar/Adversarial/GPTInst/dataset.json'
with open(input_file_path, 'r') as f:
    input_data_llmbar_gptinst = json.load(f)
    for item in input_data_llmbar_gptinst:
        prompt = build_autoj_input(prompt=item['input'], resp1=item['output_1'], resp2=item['output_2'], protocol="pairwise_tie")
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_gptinst.append(text)
outputs_llmbar_gptinst = llm.generate(texts_gptinst, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_response.json'
with open(output_file_path, 'w') as f:
    for output, item in zip(outputs_llmbar_gptinst, input_data_llmbar_gptinst):
        pred_label = extract_pariwise_result(output.outputs[0].text)
        item['response_judge'] = output.outputs[0].text
        item['pred_label'] = pred_label
        json.dump(item, f)
        f.write('\n')
        print(pred_label)

Processed prompts: 100%|██████████| 92/92 [00:12<00:00,  7.41it/s]

-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1


In [8]:
# llmbar_gptout

texts_gptout = []
input_file_path = '/root/LLMBar/Dataset/LLMBar/Adversarial/GPTOut/dataset.json'
with open(input_file_path, 'r') as f:
    input_data_llmbar_gptout = json.load(f)
    for item in input_data_llmbar_gptout:
        prompt = build_autoj_input(prompt=item['input'], resp1=item['output_1'], resp2=item['output_2'], protocol="pairwise_tie")
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_gptout.append(text)
outputs_llmbar_gptout = llm.generate(texts_gptout, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_response.json'
with open(output_file_path, 'w') as f:
    for output, item in zip(outputs_llmbar_gptout, input_data_llmbar_gptout):
        pred_label = extract_pariwise_result(output.outputs[0].text)
        item['response_judge'] = output.outputs[0].text
        item['pred_label'] = pred_label
        json.dump(item, f)
        f.write('\n')
        print(pred_label)

Processed prompts: 100%|██████████| 47/47 [00:03<00:00, 13.11it/s]

-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1


In [9]:
# llmbar_manual

texts_manual = []
input_file_path = '/root/LLMBar/Dataset/LLMBar/Adversarial/Manual/dataset.json'
with open(input_file_path, 'r') as f:
    input_data_llmbar_manual = json.load(f)
    for item in input_data_llmbar_manual:
        prompt = build_autoj_input(prompt=item['input'], resp1=item['output_1'], resp2=item['output_2'], protocol="pairwise_tie")
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_manual.append(text)
outputs_llmbar_manual = llm.generate(texts_manual, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_response.json'
with open(output_file_path, 'w') as f:
    for output, item in zip(outputs_llmbar_manual, input_data_llmbar_manual):
        pred_label = extract_pariwise_result(output.outputs[0].text)
        item['response_judge'] = output.outputs[0].text
        item['pred_label'] = pred_label
        json.dump(item, f)
        f.write('\n')
        print(pred_label)

Processed prompts: 100%|██████████| 46/46 [00:05<00:00,  8.85it/s]

-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1


In [10]:
# mt-bench-type1
import json
texts_mtbench = []
input_file_path = '/root/dataset/mt_bench_human.json'
with open(input_file_path, 'r') as f:
    for line in f:
        item = json.loads(line)
        prompt = build_autoj_input(prompt=item['conversation_a'][0]['content'],
                                   resp1=item['conversation_a'][1]['content'],
                                   resp2=item['conversation_b'][1]['content'],
                                   prompt2=item['conversation_a'][2]['content'],
                                   resp1_2=item['conversation_a'][3]['content'],
                                   resp2_2=item['conversation_b'][3]['content'],
                                   protocol="multiturn_pairwise_tie_type1")
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_mtbench.append(text)
outputs_mtbench = llm.generate(texts_mtbench, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_response_type1.json'
with open(input_file_path, 'r') as fin:
    with open(output_file_path, 'w') as f:
        for output, line in zip(outputs_mtbench, fin):
            item = json.loads(line)
            pred_label = extract_pariwise_result(output.outputs[0].text)
            item['response_judge'] = output.outputs[0].text
            item['pred_label'] = pred_label
            json.dump(item, f)
            f.write('\n')
            print(pred_label)

Processed prompts:   0%|          | 3/3355 [00:00<12:31,  4.46it/s]

WARNING 05-29 11:25:32 scheduler.py:648] Input prompt (2062 tokens) is too long and exceeds limit of 2048
WARNING 05-29 11:25:32 scheduler.py:648] Input prompt (2062 tokens) is too long and exceeds limit of 2048


Processed prompts:   0%|          | 7/3355 [00:01<07:45,  7.19it/s]

WARNING 05-29 11:25:32 scheduler.py:648] Input prompt (2677 tokens) is too long and exceeds limit of 2048
WARNING 05-29 11:25:32 scheduler.py:648] Input prompt (2677 tokens) is too long and exceeds limit of 2048


Processed prompts:   0%|          | 15/3355 [00:02<11:38,  4.78it/s]

...

WARNING 05-29 11:34:17 scheduler.py:648] Input prompt (2132 tokens) is too long and exceeds limit of 2048
WARNING 05-29 11:34:17 scheduler.py:648] Input prompt (2132 tokens) is too long and exceeds limit of 2048
WARNING 05-29 11:34:17 scheduler.py:648] Input prompt (2132 tokens) is too long and exceeds limit of 2048
WARNING 05-29 11:34:17 scheduler.py:648] Input prompt (2132 tokens) is too long and exceeds limit of 2048
WARNING 05-29 11:34:17 scheduler.py:648] Input prompt (2132 tokens) is too long and exceeds limit of 2048
WARNING 05-29 11:34:17 scheduler.py:648] Input prompt (2132 tokens) is too long and exceeds limit of 2048


Processed prompts:  98%|█████████▊| 3302/3355 [08:46<00:02, 18.11it/s]

WARNING 05-29 11:34:18 scheduler.py:648] Input prompt (2586 tokens) is too long and exceeds limit of 2048
WARNING 05-29 11:34:18 scheduler.py:648] Input prompt (2586 tokens) is too long and exceeds limit of 2048


Processed prompts: 100%|██████████| 3355/3355 [08:59<00:00,  6.22it/s]


-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-

In [11]:
"""# mt-bench-type2
import json
texts_mtbench = []
input_file_path = '/root/dataset/mt_bench_human.json'
with open(input_file_path, 'r') as f:
    for line in f:
        item = json.loads(line)
        prompt = build_autoj_input(prompt=item['conversation_a'][0]['content'],
                                   resp1=item['conversation_a'][1]['content'],
                                   resp2=item['conversation_b'][1]['content'],
                                   prompt2=item['conversation_a'][2]['content'],
                                   resp1_2=item['conversation_a'][3]['content'],
                                   resp2_2=item['conversation_b'][3]['content'],
                                   protocol="multiturn_pairwise_tie_type2")
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_mtbench.append(text)
outputs_mtbench = llm.generate(texts_mtbench, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_response_type2.json'
with open(input_file_path, 'r') as fin:
    with open(output_file_path, 'w') as f:
        for output, line in zip(outputs_mtbench, fin):
            item = json.loads(line)
            pred_label = extract_pariwise_result(output.outputs[0].text)
            item['response_judge'] = output.outputs[0].text
            item['pred_label'] = pred_label
            json.dump(item, f)
            f.write('\n')
            print(pred_label)"""

'# mt-bench-type2\nimport json\ntexts_mtbench = []\ninput_file_path = \'/root/dataset/mt_bench_human.json\'\nwith open(input_file_path, \'r\') as f:\n    for line in f:\n        item = json.loads(line)\n        prompt = build_autoj_input(prompt=item[\'conversation_a\'][0][\'content\'],\n                                   resp1=item[\'conversation_a\'][1][\'content\'],\n                                   resp2=item[\'conversation_b\'][1][\'content\'],\n                                   prompt2=item[\'conversation_a\'][2][\'content\'],\n                                   resp1_2=item[\'conversation_a\'][3][\'content\'],\n                                   resp2_2=item[\'conversation_b\'][3][\'content\'],\n                                   protocol="multiturn_pairwise_tie_type2")\n        messages = [\n        {"role": "system", "content": "You are a helpful assistant."},\n        {"role": "user", "content": prompt}\n        ]\n        text = tokenizer.apply_chat_template(\n            